# Install Required libraries

In [1]:
!pip install sacremoses transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 12.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=f7b24e148698ffd6bf1cf9900ca2bbcde10976243e8d5b3fe234af644ea1afc8
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


# Import required libraries

In [2]:
from transformers import BioGptTokenizer, BioGptForSequenceClassification, pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Import Data

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dissertation/data/final_clinicalText.csv')

# Load Model and tokenizer (BioGPT)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#GPT-2 is a model with absolute position embeddings so it's usually advised to pad the inputs on the right rather than the left. Remember this
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
tokenizer.padding_side = "left"

model = AutoModelForSequenceClassification.from_pretrained("microsoft/biogpt", num_labels=2, return_dict=True)
#model.resize_token_embeddings(len(tokenizer))

#model = model.to(device)

Some weights of BioGptForSequenceClassification were not initialized from the model checkpoint at microsoft/biogpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from peft import (
    LoraConfig,
    PeftType,
    PromptEncoderConfig,
    PeftConfig,
    PeftModel,
    PeftModelForFeatureExtraction,
    PeftModelForSequenceClassification

)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


peft_model_id = "Lukee4/test-2019"
config = PeftConfig.from_pretrained(peft_model_id)

config.task_type='FEATURE_EXTRACTION'
inference_model = AutoModel.from_pretrained("microsoft/biogpt", output_hidden_states=True)

tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

# Load the Lora model
inference_model = PeftModel.from_pretrained(inference_model, peft_model_id, output_hidden_states=True)

#model = model.to(device)

In [20]:
config

PeftConfig(peft_type='LORA', auto_mapping={'base_model_class': 'PeftModelForSequenceClassification', 'parent_library': 'peft.peft_model'}, base_model_name_or_path=None, revision=None, task_type='FEATURE_EXTRACTION', inference_mode=True)

In [6]:
inference_model

PeftModel(
  (base_model): LoraModel(
    (model): BioGptModel(
      (embed_tokens): Embedding(42384, 1024, padding_idx=1)
      (embed_positions): BioGptLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-23): 24 x BioGptDecoderLayer(
          (self_attn): BioGptAttention(
            (k_proj): Linear(
              in_features=1024, out_features=1024, bias=True
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=1024, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (v_proj): Linear(
              in_features=1024, out_features=1024, bias=True
              (lora_dr

## Define the pipeline

In [7]:
# Create the pipeline
p = pipeline(
    task="feature-extraction",
    tokenizer=tokenizer,
    model=model,
    framework="pt",
    device=0,  # use CUDA with 0
)

### Get the embeddings of the last token of the last hidden state

In [9]:
# Custom function to extract the embedding of the last token
def extract_last_token(last_hidden_states):
    last_hidden_states = np.array(last_hidden_states)
    return last_hidden_states[:,-1]

# Process the data using the pipeline
results = p([row["extracted_text"] for _, row in df.iterrows()])

# Extract the last token of the last hidden state
embeddings = [extract_last_token(hidden_state) for hidden_state in results]

In [10]:
print(results)

[[[0.3513328433036804, 0.14394423365592957]], [[0.3142324686050415, 0.2625390887260437]], [[0.2849867641925812, 0.24355460703372955]], [[0.35118088126182556, 0.150163471698761]], [[0.3466610908508301, 0.15413329005241394]], [[0.41712191700935364, 0.22166842222213745]], [[0.3420875072479248, 0.1479831486940384]], [[0.3644115924835205, 0.14957618713378906]], [[0.8649017810821533, -0.12881428003311157]], [[0.38352179527282715, 0.2698114514350891]], [[0.4110998809337616, 0.23821793496608734]], [[0.37151777744293213, 0.27125608921051025]], [[0.3470679819583893, 0.15895512700080872]], [[0.09429216384887695, 0.01721193641424179]], [[0.3967858552932739, 0.25288155674934387]], [[0.3466610908508301, 0.15413329005241394]], [[0.3008219003677368, 0.2644033432006836]], [[-0.23868262767791748, 0.20127534866333008]], [[0.35039806365966797, 0.1647031456232071]], [[0.20530959963798523, 0.11347898095846176]], [[0.36957913637161255, 0.25853294134140015]], [[0.35039806365966797, 0.1647031456232071]], [[0.3

In [13]:
# Assign the reshaped embeddings to the "embeddings" column in the DataFrame
df["emb_biogpt_WITH_FineT"] = embeddings

# Print the resulting DataFrame
print(df)

    clinical_action                                     extracted_text  \
0                 1  ATTENDANCE STATUS Did Not Attend - no advance ...   
1                 0  Your patient has been informed,  along with an...   
2                 0  Your patient has been informed,  along with an...   
3                 1  ATTENDANCE STATUS Did Not Attend - no advance ...   
4                 1  ATTENDANCE STATUS Did Not Attend - no advance ...   
..              ...                                                ...   
94                0  ATTENDANCE STATUS Attended on time or, if late...   
95                0  ATTENDANCE STATUS Attended on time or, if late...   
96                1  ATTENDANCE STATUS Attended on time or, if late...   
97                0  ATTENDANCE STATUS Attended on time or, if late...   
98                1  ATTENDANCE STATUS Did Not Attend - no advance ...   

    emb_biogpt_WITH_FineT  
0   [-0.7855649590492249]  
1    [-1.053297519683838]  
2   [-0.9226635694503784]  

## Save to csv ot txt

In [ ]:
df.to_json('/content/drive/MyDrive/Colab Notebooks/dissertation/data/letters_WITH_FT.json', orient='records')

#df.to_csv("/content/drive/MyDrive/Colab Notebooks/dissertation/data/datafinal.csv", index=False)
#np.savetxt(r"/content/drive/MyDrive/Colab Notebooks/dissertation/data/datafinal.txt", df, fmt='%s')

#For testing pusposes (SVC code)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.emb_biogpt_no_FineT, df.Predict, test_size=0.2, random_state=42)

In [ ]:
X_train = np.array(X_train.tolist())
X_train = np.squeeze(X_train)
X_test = np.array(X_test.tolist())
X_test = np.squeeze(X_test)
print(np.array(X_train).shape)

(150, 1024)


In [ ]:
X_train

In [ ]:
from sklearn.svm import SVC
svm_model = SVC(decision_function_shape='ovo')
svm_model.fit(X_train, y_train)

SVC(decision_function_shape='ovo')

In [ ]:
y_pred = svm_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           N       0.61      0.78      0.68        18
           U       0.00      0.00      0.00         3
           Y       0.60      0.53      0.56        17

    accuracy                           0.61        38
   macro avg       0.40      0.44      0.42        38
weighted avg       0.56      0.61      0.58        38



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
c = pipeline(
    task="text-classification",
    tokenizer="microsoft/biogpt",
    model=model,
    framework="pt",
    device=0,  # use CUDA with 0
)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
results2 = c([row["Chief Complaint"] for _, row in df.iterrows()])


In [ ]:
print(classification_report(, results2))

[{'label': 'LABEL_0', 'score': 0.6615974307060242}, {'label': 'LABEL_0', 'score': 0.8978613615036011}, {'label': 'LABEL_0', 'score': 0.737217903137207}, {'label': 'LABEL_1', 'score': 0.5124248266220093}, {'label': 'LABEL_0', 'score': 0.7070297598838806}, {'label': 'LABEL_0', 'score': 0.6875593066215515}, {'label': 'LABEL_0', 'score': 0.6362900137901306}, {'label': 'LABEL_1', 'score': 0.5346298813819885}, {'label': 'LABEL_0', 'score': 0.7779340147972107}, {'label': 'LABEL_0', 'score': 0.6526575088500977}, {'label': 'LABEL_0', 'score': 0.7227324843406677}, {'label': 'LABEL_0', 'score': 0.8791565895080566}, {'label': 'LABEL_1', 'score': 0.932992696762085}, {'label': 'LABEL_0', 'score': 0.6692107915878296}, {'label': 'LABEL_0', 'score': 0.5934761166572571}, {'label': 'LABEL_0', 'score': 0.5123370885848999}, {'label': 'LABEL_0', 'score': 0.7277477383613586}, {'label': 'LABEL_0', 'score': 0.9222954511642456}, {'label': 'LABEL_0', 'score': 0.6491223573684692}, {'label': 'LABEL_0', 'score': 0.

# Normalize features https://discuss.huggingface.co/t/finetuning-for-feature-extraction-i-e-unsupervised-fine-tuning/12595/*4*

## Extract the embeddings using mean pooling

In [ ]:
def mean_pooling(last_hidden_states, ):
    last_4_layers = last_hidden_states[-12:]  # Consider the last 4 layers
    return np.mean(last_4_layers, axis=1)

# Process the data using the pipeline
results = p([row["text"] for _, row in df2.iterrows()])

features = np.squeeze(results)

print(features.shape)
# Perform mean pooling on the last hidden states
embeddings = [mean_pooling(hidden_state) for hidden_state in results]

# Create a DataFrame to store the results
df2["embeddings12"] = embeddings

# Print the resulting DataFrame
print(df2)